In [11]:
import pandas as pd
import os
import numpy as np
from sklearn.cross_decomposition import CCA
from sklearn.decomposition import PCA

In [12]:
#embs = pd.read_csv('/neurospin/dico/jlaval/Output/SC-sylv_12-16/16-38-32_170/troiani_custom_embeddings/custom_cross_val_embeddings.csv')
#embs = pd.read_csv('/neurospin/dico/jlaval/Output/SC-sylv_12-16/16-39-27_30/troiani_custom_embeddings/custom_cross_val_embeddings.csv')
#embs = pd.read_csv('/neurospin/dico/jlaval/Output/SC-sylv_left_v1/keep_bottom/troiani_custom_embeddings/custom_cross_val_embeddings.csv') ## V1 before even changing cutin prop
#embs = pd.read_csv('/neurospin/dico/jlaval/Output/SC-sylv_left_v1/no_keep_bottom/troiani_custom_embeddings/custom_cross_val_embeddings.csv') ## same here = best ?
#embs = pd.read_csv('/volatile/jl277509/Runs/02_STS_babies/Program/Output/SC-sylv_isomaps/16-40-54_148/troiani_embeddings/custom_cross_val_embeddings.csv')


In [7]:
## V0 with 256 dim
for k in range(5):
    embs = pd.read_csv(f'/volatile/jl277509/Runs/02_STS_babies/Program/Output/ISOMAP_CENTRAL/22-58-04_{k}/troiani_custom_embeddings/custom_cross_val_embeddings.csv')
    embs.columns=['Subject']+embs.columns[1:].tolist()
    embs_orig = embs.drop_duplicates(subset='Subject', keep='first')
    cols_embs = [f'dim{k}' for k in range(1,257)]

    # reformat embeddings and isomap
    isomap = pd.read_csv('/neurospin/dico/data/deep_folding/current/datasets/hcp/hcp_isomap_labels.csv')
    merged = pd.merge(isomap, embs)
    cols_iso = [f'Isomap_central_left_dim{k}' for k in range(1,7)]
    embs, isomap = merged[cols_embs].to_numpy(), merged[cols_iso].to_numpy()

    # fit CCA and return score
    cca = CCA(n_components=len(cols_iso), scale=False) ## need to set scale to false !!
    cca.fit(embs, isomap)
    projected_embs = cca.transform(embs)
    print(cca.score(embs, isomap))

    ## WHY WOULD CCA BE MUCH WORSE IF EACH R2 IS SIMILAR ??

0.1791501817258981
0.16115951622098967
0.1760009660158106
0.14662874231693138
0.16832458886068602


In [17]:
cca.y_weights_

(6, 6)

In [13]:
## V0 with 32 dim PCA
embs = pd.read_csv('/neurospin/dico/data/deep_folding/current/models/Champollion_V0/SC-sylv_left/11-43-38_2/hcp_random_embeddings/full_embeddings.csv')
embs.columns=['Subject']+embs.columns[1:].tolist()
embs_orig = embs.drop_duplicates(subset='Subject', keep='first')
cols_orig = [f'dim{k}' for k in range(1,257)]
cols_embs = [f'dim{k}' for k in range(1,33)]

# perform PCA on the embeddings to keep 32 dims
pca = PCA(n_components=32)
embs = pca.fit_transform(embs_orig[cols_orig])
embs = pd.DataFrame(columns=cols_embs, data=embs)
embs['Subject']=embs_orig['Subject']

# reformat embeddings and isomap
isomap = pd.read_csv('/neurospin/dico/data/deep_folding/current/datasets/hcp/hcp_isomap_labels.csv')
merged = pd.merge(isomap, embs)
cols_iso = [f'Isomap_central_left_dim{k}' for k in range(1,7)]
embs, isomap = merged[cols_embs].to_numpy(), merged[cols_iso].to_numpy()

# fit CCA and return score
cca = CCA(n_components=len(cols_iso), scale=False) ## need to set scale to false !!
cca.fit(embs, isomap)
projected_embs = cca.transform(embs)
print(cca.score(embs, isomap))

-51.72307740713265


In [10]:
# V1

model = 'cutin_1'
epochs = [0,10,20,30]

for epoch in epochs:
    embs = pd.read_csv(f'/neurospin/dico/jlaval/Output/SC-sylv_left/{model}/hcp_isomap_random_epoch{epoch}_embeddings/full_embeddings.csv')

    # reformat embeddings and isomap
    isomap = pd.read_csv('/neurospin/dico/data/deep_folding/current/datasets/hcp/hcp_isomap_labels.csv')
    embs.columns=['Subject']+embs.columns[1:].tolist()
    merged = pd.merge(isomap, embs)
    cols_embs = [f'dim{k}' for k in range(1,33)]
    cols_iso = [f'Isomap_central_left_dim{k}' for k in range(1,7)]
    embs, isomap = merged[cols_embs].to_numpy(), merged[cols_iso].to_numpy()

    # fit CCA and return score
    cca = CCA(n_components=len(cols_iso), scale=False) ## need to set scale to false !!
    cca.fit(embs, isomap)
    projected_embs = cca.transform(embs)
    print(f'epoch {epoch} : {cca.score(embs, isomap)}')

epoch 0 : 0.06489502409476167
epoch 10 : 0.3563535884304358
epoch 20 : 0.3385936421976936
epoch 30 : 0.34471633952819664


In [ ]:
## keep it simple for the score as there are only 6 dims
## NB : results consistent with linear regression on the features
## NB : les dimensions des isomaps étant orthogonales, peut-être que la CCA n'apporte rien par rapport à des régressions linéaires sur chaque var.
# Mais c'est plus pratique d'avoir une seule valeur
# Devrait-on tenir compte de la taille d'effet de chaque isomap dans le score ? Ici on leur donne un poids identique.
# Mais la CKA est trop restrictive